In [1]:
%matplotlib inline

#### Autograd求导机制
PyTorch中所有神经网络的核心是`autograd`包。
`autograd`包为张量上的所有操作提供了自动求导。它是一个在运行时定义的框架，这意味这反向传播时根据代码来确定如何运行，并且每次迭代可以是不同的。

#### 张量（Tesnor）
`torch.Tensor`是这个包的核心类。如果设置`.requires_grad`为`True`，那么将会追踪所有对于该张量的操作。当完成计算后通过调用`.backward()`，自动计算所有的梯度，这个张量的所有梯度将会自动积累到`.grad`属性。

要阻止张量跟踪历史记录，可以调用`.detach()`方法将其与计算历史记录分离，并禁止跟踪它将来的计算记录。

为了防止跟踪历史记录（和使用内存），可以将代码块包装在`with torch.no_grad():`中。在评估模型时特别有用，因为模型可能具有`requires_grad = True`的可训练参数，但是我们不需要梯度计算。

在自动梯度计算中还有另外一个重要的类`Function`。

`Tensor`和`Function`互相连接并生成一个非循环图，它表示和存储了完整的计算历史，每个张量都有一个`.grad_fn`属性，这个属性引用了一个创建了`Tensor`的`Function`（除非这个张量时用户手动创建的，即这个张量的`grad_fn`是`None`）

如果需要计算导数，你可以在`Tensor`上调用`.backward()`。如果`Tensor`是一个标量（即它包含一个元素数据）则不需要为`backward()`指定任何参数。到那时如果它有更多的元素，你需要指定一个`gradient`参数来匹配张量的形状。

In [2]:
import torch

In [3]:
x = torch.ones(2, 2, requires_grad=True) # 创建一个2x2的张量，并设置requires_grad=True以便进行梯度计算
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [4]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [5]:
print(y.grad_fn)  # y是通过一个加法操作创建的，所以它有一个grad_fn

In [6]:
z = y * y * 3
out = z.mean()
print(z, out) # z是通过乘法和加法操作创建的，out是z的均值

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


`.requires_grad_(...)`可以改变现有张量的`requires_grad`属性。如果没有指定的话，默认输入的flag是`False`。

In [7]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)  # 默认情况下，requires_grad是False
a.requires_grad_(True)
print(a.requires_grad)  # 现在requires_grad是True
b = (a * a).sum()
print(b.grad_fn)  # b是通过一系列操作创建的，所以它有一个grad_fn

False
True


#### 梯度
反向传播
因为`out`是一个标量（scalar），`out.backward()`等于`out.backward(torch.tensor(1))`

In [8]:
out.backward()  # 反向传播，计算梯度

In [9]:
print(x.grad)  # 输出x的梯度

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


来看一个vector-Jacobian product的例子

In [10]:
x = torch.randn(3, requires_grad=True) # 创建一个3维的张量，并设置requires_grad=True
y = x * 2
while y.data.norm() < 1000: # 当y的范数小于1000时，继续循环
    y = y * 2

print(y) 

tensor([282.7868, 976.9248, 272.6789], grad_fn=<MulBackward0>)


这种情形中，`y`不再是标量。`torch.autograd`无法直接计算出完整的雅可比行列，但是如果我们只想要vector-Jacobian product，只需要将向量作为参数传入`backward`：

In [11]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float32)  # 定义一个梯度向量
y.backward(gradients)  # 计算vector-Jacobian product

print(x.grad)  # 输出x的梯度

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


如果`.requires_grad=True`但是又不希望进行autograd的计算，那么可以将变量包裹在`with torch.no_grad()`中：

In [14]:
print(x.requires_grad)  # 检查x是否需要梯度计算
print((x ** 2).requires_grad)  # 检查x的平方是否需要梯度计算

with torch.no_grad():
    print((x ** 2).requires_grad)  # 在no_grad环境下，x的平方不需要梯度计算

True
True
False
